In [1]:
import tensorflow as tf
import tensorflow.keras as K
# tf.enable_eager_execution()

In [2]:
tf.__version__

'1.10.0'

In [3]:
import numpy as np

In [4]:
# import encoders
import sys
sys.path.append('../')
import encoders

## Load data

In [5]:
# import dataset
mnist = K.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# reshape dataset
x_train = x_train.reshape([-1,28,28,1])
x_test = x_test.reshape([-1,28,28,1])
y_train = K.utils.to_categorical(y_train)
y_test = K.utils.to_categorical(y_test)

In [6]:
datagen = K.preprocessing.image.ImageDataGenerator(
                             rotation_range=5,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=False)

In [7]:
datagen.fit(x_train)

## Mnist test model

In [8]:
# defining model
input_layer = K.layers.Input(shape=(28,28,1))
output_layer = encoders.mnist_test(input_layer)
mnist_model = K.Model(inputs=input_layer, outputs=output_layer)

In [9]:
# The compile step specifies the training configuration.
mnist_model.compile(optimizer=tf.train.AdamOptimizer(),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [10]:
mnist_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              3212288   
__________

## not augmented

In [11]:
mnist_model.fit(x=x_train,y=y_train, batch_size=1024,epochs=50,
                validation_data=(x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 5s 86us/step - loss: 0.4392 - acc: 0.8662 - val_loss: 0.1001 - val_acc: 0.9686
Epoch 2/50
60000/60000 [==============================] - 3s 53us/step - loss: 0.0900 - acc: 0.9729 - val_loss: 0.0521 - val_acc: 0.9815
Epoch 3/50
60000/60000 [==============================] - 3s 54us/step - loss: 0.0570 - acc: 0.9825 - val_loss: 0.0344 - val_acc: 0.9879
Epoch 4/50
60000/60000 [==============================] - 3s 53us/step - loss: 0.0438 - acc: 0.9865 - val_loss: 0.0301 - val_acc: 0.9885
Epoch 5/50
60000/60000 [==============================] - 3s 53us/step - loss: 0.0348 - acc: 0.9891 - val_loss: 0.0311 - val_acc: 0.9888
Epoch 6/50
60000/60000 [==============================] - 3s 53us/step - loss: 0.0285 - acc: 0.9913 - val_loss: 0.0265 - val_acc: 0.9910
Epoch 7/50
60000/60000 [==============================] - 3s 54us/step - loss: 0.0242 - acc: 0.9926 - val_loss: 0.0246 - val_acc

## augmented

In [12]:
# defining model
input_layer = K.layers.Input(shape=(28,28,1))
output_layer = encoders.mnist_test(input_layer)
mnist_model = K.Model(inputs=input_layer, outputs=output_layer)

In [13]:
# The compile step specifies the training configuration.
mnist_model.compile(optimizer=tf.train.AdamOptimizer(),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [14]:
mnist_model.fit_generator(datagen.flow(x_train, y_train, batch_size=1024),epochs=50,
                          validation_data=(x_test,y_test))

Epoch 1/50
59/59 [==============================] - 12s 211ms/step - loss: 0.6450 - acc: 0.7943 - val_loss: 0.0825 - val_acc: 0.9727
Epoch 2/50
59/59 [==============================] - 12s 196ms/step - loss: 0.1339 - acc: 0.9590 - val_loss: 0.0516 - val_acc: 0.9841
Epoch 3/50
59/59 [==============================] - 12s 197ms/step - loss: 0.0910 - acc: 0.9720 - val_loss: 0.0422 - val_acc: 0.9856
Epoch 4/50
59/59 [==============================] - 12s 199ms/step - loss: 0.0737 - acc: 0.9773 - val_loss: 0.0357 - val_acc: 0.9883
Epoch 5/50
59/59 [==============================] - 12s 202ms/step - loss: 0.0608 - acc: 0.9811 - val_loss: 0.0278 - val_acc: 0.9911
Epoch 6/50
59/59 [==============================] - 12s 200ms/step - loss: 0.0513 - acc: 0.9838 - val_loss: 0.0261 - val_acc: 0.9917
Epoch 7/50
59/59 [==============================] - 12s 201ms/step - loss: 0.0459 - acc: 0.9856 - val_loss: 0.0249 - val_acc: 0.9918
Epoch 8/50
59/59 [==============================] - 12s 201ms/step - 

## Enet mnist model

In [27]:
# input layer
input_layer_enet = tf.keras.layers.Input(shape=(28,28,1))

In [28]:
# mnist model enet
output_layer_enet = encoders.enet_encoder_mnist(input_layer_enet,dropout=0.25)
mnist_model_enet = tf.keras.Model(inputs=input_layer_enet, outputs=output_layer_enet)

In [29]:
mnist_model_enet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 30, 30, 1)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 14, 14, 15)   150         zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 14, 14, 1)    0           input_5[0][0]                    
__________________________________________________________________________________________________
concatenat

In [30]:
# lr_scheduler
callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./logs',batch_size=1024)]

In [31]:
# The compile step specifies the training configuration.
mnist_model_enet.compile(optimizer=tf.train.AdamOptimizer(learning_rate=3e-4),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

In [33]:
mnist_model_enet.fit(x=x_train,y=y_train, batch_size=1024,epochs=50,
                     validation_data=(x_test,y_test),
                    callbacks=callbacks)

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 12s 204us/step - loss: 0.0318 - acc: 0.9897 - val_loss: 0.0280 - val_acc: 0.9905
Epoch 2/50
60000/60000 [==============================] - 12s 204us/step - loss: 0.0306 - acc: 0.9902 - val_loss: 0.0285 - val_acc: 0.9906
Epoch 3/50
60000/60000 [==============================] - 12s 204us/step - loss: 0.0308 - acc: 0.9900 - val_loss: 0.0330 - val_acc: 0.9899
Epoch 4/50
60000/60000 [==============================] - 12s 204us/step - loss: 0.0294 - acc: 0.9905 - val_loss: 0.0310 - val_acc: 0.9901
Epoch 5/50
60000/60000 [==============================] - 12s 204us/step - loss: 0.0295 - acc: 0.9904 - val_loss: 0.0280 - val_acc: 0.9907
Epoch 6/50
60000/60000 [==============================] - 12s 204us/step - loss: 0.0276 - acc: 0.9911 - val_loss: 0.0298 - val_acc: 0.9896
Epoch 7/50
60000/60000 [==============================] - 12s 205us/step - loss: 0.0281 - acc: 0.9913 - val_loss: 0.

## augmented

In [35]:
# mnist model enet
output_layer_enet = encoders.enet_encoder_mnist(input_layer_enet,dropout=0.20)
mnist_model_enet = tf.keras.Model(inputs=input_layer_enet, outputs=output_layer_enet)

In [36]:
mnist_model_enet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 30, 30, 1)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 14, 14, 15)   150         zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  (None, 14, 14, 1)    0           input_5[0][0]                    
__________________________________________________________________________________________________
concatenat

In [37]:
# lr_scheduler
callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./logs',batch_size=1024)]

In [39]:
# The compile step specifies the training configuration.
mnist_model_enet.compile(optimizer=tf.train.AdamOptimizer(learning_rate=3e-4),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

In [42]:
mnist_model_enet.fit_generator(datagen.flow(x_train, y_train, batch_size=1024),epochs=200,
                               validation_data=(x_test,y_test),
                               callbacks=callbacks)

Epoch 1/100
59/59 [==============================] - 13s 223ms/step - loss: 0.0109 - acc: 0.9965 - val_loss: 0.0240 - val_acc: 0.9922
Epoch 2/100
59/59 [==============================] - 13s 218ms/step - loss: 0.0093 - acc: 0.9971 - val_loss: 0.0197 - val_acc: 0.9937
Epoch 3/100
59/59 [==============================] - 13s 216ms/step - loss: 0.0098 - acc: 0.9967 - val_loss: 0.0157 - val_acc: 0.9950
Epoch 4/100
59/59 [==============================] - 12s 208ms/step - loss: 0.0100 - acc: 0.9970 - val_loss: 0.0146 - val_acc: 0.9951
Epoch 5/100
59/59 [==============================] - 12s 210ms/step - loss: 0.0102 - acc: 0.9964 - val_loss: 0.0163 - val_acc: 0.9951
Epoch 6/100
59/59 [==============================] - 13s 213ms/step - loss: 0.0105 - acc: 0.9966 - val_loss: 0.0156 - val_acc: 0.9948
Epoch 7/100
59/59 [==============================] - 13s 212ms/step - loss: 0.0097 - acc: 0.9969 - val_loss: 0.0215 - val_acc: 0.9936
Epoch 8/100
59/59 [==============================] - 13s 214ms

Epoch 62/100
59/59 [==============================] - 13s 215ms/step - loss: 0.0077 - acc: 0.9973 - val_loss: 0.0193 - val_acc: 0.9943
Epoch 63/100
59/59 [==============================] - 12s 210ms/step - loss: 0.0077 - acc: 0.9975 - val_loss: 0.0195 - val_acc: 0.9935
Epoch 64/100
59/59 [==============================] - 12s 209ms/step - loss: 0.0077 - acc: 0.9975 - val_loss: 0.0156 - val_acc: 0.9950
Epoch 65/100
59/59 [==============================] - 13s 217ms/step - loss: 0.0070 - acc: 0.9974 - val_loss: 0.0183 - val_acc: 0.9947
Epoch 66/100
59/59 [==============================] - 13s 213ms/step - loss: 0.0081 - acc: 0.9973 - val_loss: 0.0147 - val_acc: 0.9958
Epoch 67/100
59/59 [==============================] - 12s 212ms/step - loss: 0.0082 - acc: 0.9975 - val_loss: 0.0169 - val_acc: 0.9951
Epoch 68/100
59/59 [==============================] - 13s 218ms/step - loss: 0.0073 - acc: 0.9975 - val_loss: 0.0180 - val_acc: 0.9948
Epoch 69/100
59/59 [==============================] - 1